In [49]:
import sys

# pandas读写csv文件
import pandas as pd
import numpy as np

path = '/Users/mingjue/downloads/'

df_train_1 = pd.read_csv(path + 'data_format1/train_format1.csv', dtype={'user_id': 'str', 'merchant_id': 'str', 'label': 'float'})
df_test_1 = pd.read_csv(path + 'data_format1/test_format1.csv', dtype={'user_id': 'str', 'merchant_id': 'str', 'label': 'float'})
df_user_info = pd.read_csv(path + 'data_format1/user_info_format1.csv', dtype='str')
df_user_log = pd.read_csv(path + 'data_format1/user_log_format1.csv', dtype='str')

df_train_2 = pd.read_csv(path + 'data_format2/train_format2.csv', dtype='str')
df_test_2 = pd.read_csv(path + 'data_format2/test_format2.csv', dtype='str')

df_infer = pd.read_csv(path + 'sample_submission.csv', dtype='str')


In [ ]:
# df_infer = pd.read_csv(path + 'sample_submission.csv')
# # 简单的预测，随机大于0小于1的小数，比如0.5、0.8，均值0.2
# df_infer['prob'] = np.random.rand(len(df_infer))
# # 保存结果到prediction.csv文件中
# df_infer.to_csv(path + 'prediction.csv', index=False)

In [35]:
df_user_info.describe(include='all') # 424170

,user_id,age_range,gender
count,424170,421953,417734
unique,424170,9,3
top,376517,3,0
freq,1,111654,285638


In [ ]:
# 检查数据
df_user_info.describe(include='all')

,user_id,age_range,gender
count,424170,421953,417734
unique,424170,9,3
top,376517,3,0
freq,1,111654,285638


In [33]:
df_user_info.groupby('age_range').size().reset_index(name='count')

,age_range,count
0,0,92914
1,1,24
2,2,52871
3,3,111654
4,4,79991
5,5,40777
6,6,35464
7,7,6992
8,8,1266


In [34]:
df_user_info.groupby('gender').size().reset_index(name='count')

,gender,count
0,0,285638
1,1,121670
2,2,10426


In [ ]:
# age_range 和 gender 有缺失值
# age_range 的取值为0-8，0表示缺失值（age_range一般对应年龄大小顺序，根据分布判断0表示未知）
# gender 的取值为0-2，2表示缺失值(根据官网描述，2和NULL表示未知)

In [50]:
# 将df_user_info的 age_range 列  字段转换为float， 取值0都替换为空值,
df_user_info['age_range'] = df_user_info['age_range'].astype(float)
df_user_info['age_range'] = df_user_info['age_range'].replace(0, np.nan)
# 将df_user_info的 gender 列 中 取值为2替换为空值
df_user_info['gender'] = df_user_info['gender'].replace('2', np.nan)
df_user_info.describe(include='all') # 424170

,user_id,age_range,gender
count,424170,329039.00000,407308
unique,424170,NaN,2
top,376517,NaN,0
freq,1,NaN,285638
mean,NaN,3.75771,NaN
std,NaN,1.31601,NaN
min,NaN,1.00000,NaN
25%,NaN,3.00000,NaN
50%,NaN,3.00000,NaN
75%,NaN,5.00000,NaN


In [56]:
df_train = df_train_1.merge(df_user_info, on='user_id', how='left')
df_train['gender'] = df_train['gender'].astype(str)
df_test = df_test_1.merge(df_user_info, on='user_id', how='left')
df_test['gender'] = df_test['gender'].astype(str)

In [52]:
df_train.describe(include='all')

,user_id,merchant_id,label,age_range,gender
count,260864,260864,260864.000000,203802.000000,250170
unique,212062,1993,NaN,NaN,2
top,221133,4044,NaN,NaN,0
freq,18,3379,NaN,NaN,176414
mean,NaN,NaN,0.061151,3.768403,NaN
std,NaN,NaN,0.239607,1.298597,NaN
min,NaN,NaN,0.000000,1.000000,NaN
25%,NaN,NaN,0.000000,3.000000,NaN
50%,NaN,NaN,0.000000,4.000000,NaN
75%,NaN,NaN,0.000000,5.000000,NaN


In [62]:
# df_train catboost 二分类 训练推理
# 固定随机种子，保证结果可复现
np.random.seed(42)
# 训练代码 
from catboost import CatBoostClassifier, Pool  
from sklearn.model_selection import train_test_split
features = ['age_range', 'gender']
target = 'label'
X = df_train[features]
y = df_train[target].astype(float)
X_test = df_test[features]        
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
train_pool = Pool(X_train, y_train, cat_features=['gender'])
val_pool = Pool(X_val, y_val, cat_features=['gender'])  
model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, eval_metric='AUC', random_seed=42, logging_level='Verbose', use_best_model=True)
model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=50, verbose=100)
y_pred = model.predict_proba(X_test)[:, 1]
df_test['prob'] = y_pred
df_test[['user_id', 'merchant_id', 'prob']].to_csv(path + 'prediction.csv', index=False)  
print(path + 'prediction.csv saved!')


0:	test: 0.5129901	best: 0.5129901 (0)	total: 30.6ms	remaining: 30.6s
100:	test: 0.5366956	best: 0.5366956 (99)	total: 2.96s	remaining: 26.4s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.5392384126
bestIteration = 143

Shrink model to first 144 iterations.
/Users/mingjue/downloads/prediction.csv saved!
